In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 85% 73.0M/85.7M [00:00<00:00, 48.1MB/s]
100% 85.7M/85.7M [00:01<00:00, 84.2MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 7.8 MB/s 
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 6.4 MB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 39.5 MB/s 
     |████████████████████████████████| 840 kB 8.7 MB/s 
     |████████████████████████████████| 120 kB 69.2 MB/s 
     |████████████████████████████████| 234 kB 45.6 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
     |████████████████████████████████| 3.4 MB 40.3 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 352 kB 59.9 MB/s 
     |████████████████████████████████| 462 kB 49.5 MB/s 
     |████████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [10]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [11]:
spec = model_spec.get('efficientdet_lite3')

In [12]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/10
397/397 [==============================] - 266s 512ms/step - det_loss: 0.9525 - cls_loss: 0.7506 - box_loss: 0.0040 - reg_l2_loss: 0.0938 - loss: 1.0462 - learning_rate: 0.0078 - gradient_norm: 3.5614 - val_det_loss: 0.5037 - val_cls_loss: 0.4330 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0941 - val_loss: 0.5978
Epoch 2/10
397/397 [==============================] - 200s 504ms/step - det_loss: 0.5731 - cls_loss: 0.4726 - box_loss: 0.0020 - reg_l2_loss: 0.0943 - loss: 0.6674 - learning_rate: 0.0070 - gradient_norm: 3.5773 - val_det_loss: 0.3866 - val_cls_loss: 0.3398 - val_box_loss: 9.3620e-04 - val_reg_l2_loss: 0.0945 - val_loss: 0.4811
Epoch 3/10
397/397 [==============================] - 200s 503ms/step - det_loss: 0.5049 - cls_loss: 0.4239 - box_loss: 0.0016 - reg_l2_loss: 0.0946 - loss: 0.5996 - learning_rate: 0.0061 - gradient_norm: 3.4142 - val_det_loss: 0.3647 - val_cls_loss: 0.3273 - val_box_loss: 7.4922e-04 - val_reg_l2_loss: 0.0947 - val_loss: 0.4595
Epoch 4/10
397

In [13]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 2145.5916378498077


In [14]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 14s 130ms/step

{'AP': 0.7592346, 'AP50': 0.89143574, 'AP75': 0.83172977, 'APs': -1.0, 'APm': 0.58194244, 'APl': 0.7716284, 'ARmax1': 0.7205888, 'ARmax10': 0.83053833, 'ARmax100': 0.83863795, 'ARs': -1.0, 'ARm': 0.6390476, 'ARl': 0.85107815, 'AP_/10': 0.69898134, 'AP_/20': 0.81452394, 'AP_/50': 0.6936544, 'AP_/100': 0.8199153, 'AP_/200': 0.8389262, 'AP_/500': 0.77747947, 'AP_/2000': 0.67116135}


In [15]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 17.566981315612793


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [16]:
model.export(export_dir='/content/')